# Installation and Repository cloning

In [8]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
#!pip install adversarial-robustness-toolbox -U
#!pip install git+https://github.com/RobustBench/robustbench.git
#!pip install multiprocess
#!pip install matplotlib
#!pip install pillow
#!pip install foolbox
#!pip install numba

## Import Libraries

In [9]:
import torch
import utils
import adversarial_attack.attack_utils as attack_utils

device = 'cpu' #torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load and Prepare Adversarial Model

In [10]:
model = 'Wang2023Better_WRN-28-10' # MainiAVG, MainiMSD, CroceL1, Wang2023Better_WRN-28-10, Wang2023Better_WRN-28-10
net, art_net, fb_net, alias = utils.get_model(modelname=model, norm='L2') # specify the norm in case of robustbench models

## Load the dataset

In [11]:
splitsize = 100 # full, int: splitsize
xtest, ytest = utils.load_dataset(dataset_split=splitsize)

Files already downloaded and verified


## Test Accuracy

In [12]:
utils.test_accuracy(net, xtest, ytest)


Accuracy of the test set is: 94.000%



## Parameters 

In [13]:
Experiment = attack_utils.Experiment_class(art_net, fb_net, net, xtest, ytest, alias,
                        epsilon_l1 = 12, 
                        epsilon_l2= 0.5, 
                        eps_iter = 0.1,  
                        norm = 1,  # 1, 2, np.inf
                        max_iterations =100, 
                        verbose=False)

# Hyperparameter Sweep Experiment

In [14]:
#hyperparameter sweep. Pick only one model.
attack_type = 'exp_attack_l1'
hyperparameter_range=[1.0]
hyperparameter = 'learning_rate' #'beta'

results_dict_hyperparameter_sweep = Experiment.hyperparameter_sweep(hyperparameter=hyperparameter, range=hyperparameter_range, attack_type=attack_type)

		-------------- Hyperparameter Sweep for Attack: exp_attack_l1: learning_rate = 1.0 ----------------

20 images done. Current Adversarial Accuracy (L1 / L2/ EN): 35.0 / 95.0/35.0%


KeyboardInterrupt: 

In [ ]:
#Attack comparison
attack_types = [#'fast_gradient_method',
                #'projected_gradient_descent',
                #'pgd_early_stopping',
                #'deep_fool',
                #'brendel_bethge',
                #'carlini_wagner_l2',
                #'elastic_net',
                #'exp_attack',
                #'exp_attack_smooth',
                #'exp_attack_l1_l2',
                #'auto_projected_gradient_descent',
                #'elastic_net_L1_rule',
                #'elastic_net_L1_rule_higher_beta',
                #'ART_AutoAttack',
                #'original_AutoAttack',
                'exp_attack_l1',
                'original_AutoAttack_apgd-only'
                ]

results_dict_attack_comparison = Experiment.attack_comparison(attack_types)